#**DBSCAN**

DBSCAN differs from the K-Means and K-Mediods algoritm in so far as it a density based algorithm as opposed to a partion approach. The DBSCAN algorithm views clusters as areas of high density separated by areas of low density and tries to avoid including "outliers" in clusters. A really nice explaination of the algorithm can be found on the [Scikit learn site](https://scikit-learn.org/stable/modules/clustering.html#dbscan).

The following definitions are useful and will help you understand the algorithm.

>* Density is defined as the number of points within a specified radious,

>* A point is a core point if it has  more  than a specified number of points (MinPts) within Eps. These are points that are at the interior of a cluster.

>* A border point has  fewer  than MinPts within Eps,  but in the neighborhood of a core point

>* A noise  point is any  point that is not a core point or a border point.

![](https://www.computing.dcu.ie/~amccarren/mcm_images/dbscan_2.png)
Figure 1: DBSCAN implementation

>* Label all points as core, border, or noise points.
>* Eliminate noise points.
>* Put an edge between all core points that are within $\epsilon$ of each other.
>* Make each group of connected core points into a separate cluster.
>* Assign each border point to one of the clusters of its associated core points.


**Advantages**
>* Can discover arbitrarily shaped clusters
>* Can find a cluster completely surronded by different clusters
>* Robust toward outlier detection
>* Insentitive o the ordering of the points in the database

**Disadvantages**
>* DBSCAN Does not work well when dealing with clusters of varying densities.

>* While DBSCAN is great at separating high density clusters from low density clusters, DBSCAN struggles with clusters of similar density.

>* Struggles with high dimensionality data.


There are other density based approaches such as OPTICS, Ankerst et al (SIGMOD'99) and DENCLUE, Hinneburg & Keim (KDD'98).
We will now implement an example which comes fro scikit learn.




As usual import the relevant library.

In [1]:
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


The next code snipet creates a data blob from sckit learn, and then standardizes the data.

In [ ]:
# #############################################################################
# Generate sample data
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                            random_state=0)

X = StandardScaler().fit_transform(X)
print(X)

We now implement the DBSCAN algorithm. Note the values of "eps" and "min_samples". These correspond to the maximum radius of the neighbourhood and the minimum number of points in an "eps" neighbourhood of that point. If you play with these you should see the number of clusters generated change.

In [ ]:



# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

We can see from the output that the algorithm works pretty well.How sensitive is the algorithm to changes in $\epsilon$ and the minimum number of points?

As usual leave your thoughts on the comments board.
